In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
#import graphviz 
import os
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
#import pydotplus
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import scipy.stats as stats
os.environ["PATH"] += os.pathsep + 'D:/Trinity_DS/Dissertations/201901/Datasets/release/bin'

D:\Installed_Applications\Anaconda\envs\Yelp\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
path="export.csv"
final_df = pd.read_csv(path,low_memory=False)

In [3]:

final_df=final_df[['day_of_week','reviews_dateAdded_hour','reviews.rating']]

In [4]:
final_df

,day_of_week,reviews_dateAdded_hour,reviews.rating
0,Monday,23,5.0
1,Monday,23,5.0
2,Monday,23,5.0
3,Monday,23,4.0
4,Monday,23,5.0
5,Monday,23,5.0
6,Monday,23,4.0
7,Monday,23,5.0
8,Monday,23,5.0
9,Monday,23,5.0


In [5]:
newdf=final_df.groupby(['day_of_week','reviews.rating']).count()
np_matix= np.array(final_df)
np_matix

array([['Monday', 23, 5.0],
       ['Monday', 23, 5.0],
       ['Monday', 23, 5.0],
       ...,
       ['Sunday', 8, 5.0],
       ['Sunday', 8, 4.0],
       ['Tuesday', 23, 5.0]], dtype=object)

In [6]:
days_rating=pd.crosstab(np_matix[:,0],np_matix[:,2],
            rownames=['Days'], colnames=['Ratings'],)
days_rating


Ratings,1.0,2.0,3.0,4.0,5.0
Days,,,,,
Friday,0,3,5,52,86
Monday,15,8,51,307,722
Saturday,0,0,0,3,7
Sunday,177,189,752,3988,7884
Thursday,0,0,0,2,6
Tuesday,14,21,56,525,2651
Wednesday,63,59,217,1314,4862


In [7]:
chi2 , p ,dof ,expected = stats.chi2_contingency(days_rating)
print("Reviews and DaysOfWeek chi_2 value---------------",chi2)
print("Reviews and DaysOfWeek p value-------------------",p)
print("Reviews and DaysOfWeek degreeoffreedom value-----",dof)


Reviews and DaysOfWeek chi_2 value--------------- 733.3213679705559
Reviews and DaysOfWeek p value------------------- 2.4001104081774692e-139
Reviews and DaysOfWeek degreeoffreedom value----- 24


# Interpreting chi^2 test of Ratings with Days of week

In [8]:
prob=0.95    
critical_day = stats.chi2.ppf(prob, dof)
if abs(chi2) >= critical_day:
    print('Dependent (reject H0)')
    print('Reviews Dependent on Days of the Week')
else:
    print('Independent (fail to reject H0)')
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

Dependent (reject H0)
Reviews Dependent on Days of the Week
significance=0.050, p=0.000
Dependent (reject H0)


# Interpreting chi^2 test of Ratings with Hours

In [9]:
hours_rating=pd.crosstab(np_matix[:,1],np_matix[:,2],
            rownames=['hours'], colnames=['Ratings'],)
chi2_2 , p_2 ,dof_2 ,expected_2 = stats.chi2_contingency(hours_rating)
hours_rating


Ratings,1.0,2.0,3.0,4.0,5.0
hours,,,,,
0,0,0,0,0,8
1,78,74,277,1464,2911
2,21,21,99,518,980
3,40,39,201,917,1662
4,7,9,34,147,259
5,52,41,170,1249,4572
6,10,17,48,378,907
7,20,29,77,368,967
8,0,0,0,2,7


In [10]:
print("Reviews and hours chi_2 value---------------",chi2_2)
print("Reviews and hours p value-------------------",p_2)
print("Reviews and hours degreeoffreedom value-----",dof_2)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_2)
if abs(chi2_2) >= critical_hour:
    print('Dependent (reject H0)')
    print('Reviews Dependent on Days of the Week')
else:
    print('Independent (fail to reject H0)')
if p_2 <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

Reviews and hours chi_2 value--------------- 886.690992130144
Reviews and hours p value------------------- 3.3841459580848884e-130
Reviews and hours degreeoffreedom value----- 92
Dependent (reject H0)
Reviews Dependent on Days of the Week
Dependent (reject H0)


In [11]:
final_df

,day_of_week,reviews_dateAdded_hour,reviews.rating
0,Monday,23,5.0
1,Monday,23,5.0
2,Monday,23,5.0
3,Monday,23,4.0
4,Monday,23,5.0
5,Monday,23,5.0
6,Monday,23,4.0
7,Monday,23,5.0
8,Monday,23,5.0
9,Monday,23,5.0


# Dimension Reduction

In [12]:
transformed_df=final_df
transformed_df['Six_Hrs_Bands'] = np.where(transformed_df['reviews_dateAdded_hour']<=6, '0-6',
                                          np.where(transformed_df['reviews_dateAdded_hour']<=12, '7-12',
                                          np.where(transformed_df['reviews_dateAdded_hour']<=18, '13-18',
                                          np.where(transformed_df['reviews_dateAdded_hour']<=24, '19-24','9999'))))
transformed_df['Weekend_WeekDays'] = np.where((transformed_df['day_of_week']=='Sunday') | (transformed_df['day_of_week']=='Saturday'),
                                              'Weekend','Weekday')
transformed_df['high_low_Rating'] = np.where(transformed_df['reviews.rating']<=4,
                                              'low','high')
transformed_df['Mix_Days'] = np.where((transformed_df['day_of_week']=='Monday') | (transformed_df['day_of_week']=='Tuesday') 
                                      | (transformed_df['day_of_week']=='Wednesday') | (transformed_df['day_of_week']=='Thursday'),
                                              'MonTuesWedThur','FriSatSun')
transformed_df['Mix_Days_v2'] = np.where((transformed_df['day_of_week']=='Monday')| (transformed_df['day_of_week']=='Friday') 
                                      | (transformed_df['day_of_week']=='Saturday')
                                      | (transformed_df['day_of_week']=='Sunday'),'MonFriSatSun','TueWedThur')
transformed_df

,day_of_week,reviews_dateAdded_hour,reviews.rating,Six_Hrs_Bands,Weekend_WeekDays,high_low_Rating,Mix_Days,Mix_Days_v2
0,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
1,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
2,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
3,Monday,23,4.0,19-24,Weekday,low,MonTuesWedThur,MonFriSatSun
4,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
5,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
6,Monday,23,4.0,19-24,Weekday,low,MonTuesWedThur,MonFriSatSun
7,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
8,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun
9,Monday,23,5.0,19-24,Weekday,high,MonTuesWedThur,MonFriSatSun


In [13]:
transformed_df.describe()
transformed_df.groupby('high_low_Rating').count()

,day_of_week,reviews_dateAdded_hour,reviews.rating,Six_Hrs_Bands,Weekend_WeekDays,Mix_Days,Mix_Days_v2
high_low_Rating,,,,,,,
high,16218,16218,16218,16218,16218,16218,16218
low,7821,7821,7821,7821,7821,7821,7821


In [14]:
transformedMatrix= np.array(transformed_df)
crSixHrBandHighLow=pd.crosstab(transformedMatrix[:,5],transformedMatrix[:,3],
            rownames=['high_low_Rating'], colnames=['Six_Hrs_Bands'])
crSixHrBandHighLow

Six_Hrs_Bands,0-6,13-18,19-24,7-12
high_low_Rating,,,,
high,11299,684,3136,1099
low,5911,442,902,566


In [15]:
chi2_v3 , p_v3 ,dof_v3 ,expected_v3 = stats.chi2_contingency(crSixHrBandHighLow)
print("HighLowRating and fourHrsBands chi_2 value---------------",chi2_v3)
print("HighLowRating and fourHrsBands p value-------------------",p_v3)
print("HighLowRating and fourHrsBands degreeoffreedom value-----",dof_v3)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_v3)
if abs(chi2_v3) >= critical_hour:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
    print('High Low Ratings Dependent on four hours bands')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
if p_v3 <= alpha:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')


HighLowRating and SixHrsBands chi_2 value--------------- 241.7921796767422
HighLowRating and SixHrsBands p value------------------- 3.8988957396689316e-52
HighLowRating and SixHrsBands degreeoffreedom value----- 3
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
High Low Ratings Dependent on Six hours bands
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5


In [16]:
crWeekendWeekdays=pd.crosstab(transformedMatrix[:,5],transformedMatrix[:,4],
            rownames=['high_low_Rating'], colnames=['WeekendWeekdays'])
crWeekendWeekdays

WeekendWeekdays,Weekday,Weekend
high_low_Rating,,
high,8327,7891
low,2712,5109


In [17]:
crWeekendWeekdays=pd.crosstab(transformedMatrix[:,4],transformedMatrix[:,5],
            rownames=['high_low_Rating'], colnames=['WeekendWeekdays'])
crWeekendWeekdays
chi2_v4 , p_v4  ,dof_v4  ,expected_v4  = stats.chi2_contingency(crWeekendWeekdays)
print("HighLowRating and WeekendWeekdays chi_2 value---------------",chi2_v4)
print("HighLowRating and WeekendWeekdays p value-------------------",p_v4)
print("HighLowRating and WeekendWeekdays degreeoffreedom value-----",dof_v4)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_v4)
if abs(chi2_v4) >= critical_hour:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
    print('High Low Ratings Dependent on four hours bands')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
if p_v4 <= alpha:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')

HighLowRating and WeekendWeekdays chi_2 value--------------- 589.6471993932211
HighLowRating and WeekendWeekdays p value------------------- 2.989840513904782e-130
HighLowRating and WeekendWeekdays degreeoffreedom value----- 1
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
High Low Ratings Dependent on Six hours bands
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5


In [27]:
crMixkdaysV1=pd.crosstab(transformedMatrix[:,6],transformedMatrix[:,5],
            rownames=['high_low_Rating'], colnames=['Mix_Days'])
crMixkdaysV1

Mix_Days,high,low
high_low_Rating,,
FriSatSun,7977,5169
MonTuesWedThur,8241,2652


In [28]:
chi2_v5 , p_v5  ,dof_v5  ,expected_v5  = stats.chi2_contingency(crMixkdaysV1)
print("HighLowRating and MixDaysofWeek chi_2 value---------------",chi2_v5)
print("HighLowRating and MixDaysofWeek p value-------------------",p_v5)
print("HighLowRating and MixDaysofWeek degreeoffreedom value-----",dof_v5)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_v5)
if abs(chi2_v5) >= critical_hour:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
    print('High Low Ratings Dependent on four hours bands')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
if p_v5 <= alpha:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')

HighLowRating and MixDaysofWeek chi_2 value--------------- 607.8390822156047
HighLowRating and MixDaysofWeek p value------------------- 3.3018286485638307e-134
HighLowRating and MixDaysofWeek degreeoffreedom value----- 1
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
High Low Ratings Dependent on Six hours bands
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5


In [29]:
crMixkdaysV2=pd.crosstab(transformedMatrix[:,7],transformedMatrix[:,5],
            rownames=['high_low_Rating'], colnames=['Mix_Days'])
crMixkdaysV2

Mix_Days,high,low
high_low_Rating,,
MonFriSatSun,8699,5550
TueWedThur,7519,2271


In [30]:
chi2_v6 , p_v6  ,dof_v6  ,expected_v6  = stats.chi2_contingency(crMixkdaysV2)
print("HighLowRating and MixDaysofWeek_2 chi_2 value---------------",chi2_v6)
print("HighLowRating and MixDaysofWeek_2 p value-------------------",p_v6)
print("HighLowRating and MixDaysofWeek_2 degreeoffreedom value-----",dof_v6)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_v5)
if abs(chi2_v6) >= critical_hour:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
    print('High Low Ratings Dependent on four hours bands')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
if p_v6 <= alpha:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')

HighLowRating and MixDaysofWeek_2 chi_2 value--------------- 655.3497222242615
HighLowRating and MixDaysofWeek_2 p value------------------- 1.5334134922714194e-144
HighLowRating and MixDaysofWeek_2 degreeoffreedom value----- 1
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
High Low Ratings Dependent on Six hours bands
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
